In [2]:
pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the GPU
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Use the CPU
    print("No GPU available, using CPU.")

No GPU available, using CPU.


In [4]:
# Create tensors on the GPU
a = torch.tensor([1.0, 2.0, 3.0], device=device)
b = torch.tensor([4.0, 5.0, 6.0], device=device)

# Perform operations on the GPU
c = a * b

In [6]:
c

tensor([ 4., 10., 18.])

In [8]:
import numpy as np
from numba import roc

# Define some kernels
@roc.jit(device=True)
def add_kernel(x, y):
    return x + y

@roc.jit
def parallel_add(x, y, out):
    i = roc.get_global_id(0)
    out[i] = add_kernel(x[i], y[i])

n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host
out = np.empty_like(x)

# Create ROCm device arrays
d_x = roc.to_device(x)
d_y = roc.to_device(y)
d_out = roc.to_device(out)

# Launch the kernel
threads_per_block = 128
blocks_per_grid = (n + threads_per_block - 1) // threads_per_block
parallel_add[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
roc.synchronize()

# Copy the result back to the host
result = d_out.copy_to_host()

# Output result
print(result)

HsaSupportError: Error at driver init: 
NUMBA_HSA_DRIVER /opt/rocm/lib/libhsa-runtime64.so is not a valid file path.  Note it must be a filepath of the .so/.dll/.dylib or the driver: